In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms
import torchreid
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

/home/lr/Documents/github/stick-it-ai/venv/lib64/python3.13/site-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


device(type='cuda', index=0)

In [2]:
from PIL import Image, ImageOps
import os

def resize_images():
    # Directory containing the images
    image_dir = 'annotated_images/pins'

    # Directory to save resized images (optional)
    resized_dir = 'reid-data/annotated_images/pins'
    os.makedirs(resized_dir, exist_ok=True)

    # Resize dimensions
    width = 128
    height = int(width * 4 / 3)  # Maintain 3:4 horizontal aspect ratio

    # Iterate through all images in the directory
    for filename in os.listdir(image_dir):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            img_path = os.path.join(image_dir, filename)
            with Image.open(img_path) as img:
                # Correct orientation based on EXIF data
                img = ImageOps.exif_transpose(img)
                # Resize the image
                resized_img = img.resize((width, height))
                # Save the resized image
                resized_img.save(os.path.join(resized_dir, filename))

    print("All images have been resized and saved with correct orientation.")
# resize_images()

In [3]:
import random
from collections import defaultdict

import os.path as osp
class StickerDataset(torchreid.data.ImageDataset):
    dataset_dir = ''
    csv_file = 'annotated_data.csv'

    def __init__(self, root='', **kwargs):
        self.root = osp.abspath(osp.expanduser(root))
        csv_path = osp.join(self.root, self.csv_file)

        df = pd.read_csv(csv_path)

        # Generate camid mapping
        user_ids = df['user_id'].unique()
        self.user_id_to_camid = {uid: idx for idx, uid in enumerate(user_ids)}

        data = []
        for _, row in df.iterrows():
            img_path = row['image_path']
            if not osp.isabs(img_path):
                img_path = osp.join(self.root, img_path)
            group_id = row['group_id']
            group_name = row['group_name']
            camid = self.user_id_to_camid[row['user_id']]
            data.append((img_path, group_id, camid, group_name))

        pid_dict = defaultdict(list)
        for img_path, group_id, camid, group_name in data:
            pid_dict[group_id].append((img_path, group_id, camid, group_name))

        train, query, gallery = [], [], []
        random.seed(42)
        for group_id, items in pid_dict.items():
            if len(items) > 2:
                random.shuffle(items)
                query_candidate = None
                gallery_candidate = None
                for item in items:
                    if query_candidate is None:
                        query_candidate = item
                    elif gallery_candidate is None and item[2] != query_candidate[2]:
                        gallery_candidate = item
                        break
                if query_candidate and gallery_candidate:
                    query.append(query_candidate)
                    gallery.append(gallery_candidate)
                    train.extend([item for item in items if item not in [query_candidate, gallery_candidate]])
                else:
                    query.append(items[0])
                    train.extend(items[1:])
        
        all_data = train + query + gallery
        all_group_ids = sorted({item[1] for item in all_data})
        self.group_id_to_pid = {gid: idx for idx, gid in enumerate(all_group_ids)}

        # Map group_name to pid
        self.pid_to_group_name = {self.group_id_to_pid[gid]: next(item[3] for item in all_data if item[1] == gid) for gid in all_group_ids}

        def map_pid(data):
            return [(img_path, self.group_id_to_pid[group_id], camid) for img_path, group_id, camid, group_name in data]

        train = map_pid(train)
        query = map_pid(query)
        gallery = map_pid(gallery)

        super(StickerDataset, self).__init__(train, query, gallery, **kwargs)

torchreid.data.register_image_dataset('sticker_dataset', dataset=StickerDataset)

In [4]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='sticker_dataset'
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded StickerDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    42 |     6494 |       300
  query    |    42 |       42 |        34
  gallery  |    30 |       30 |        25
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded StickerDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    42 |     6494 |       300
  query    |    42 |       42 |        34
  gallery  |    30 |       30 |        25
  -----

In [5]:
new_dataset = StickerDataset(root='reid-data')
train_list = new_dataset.train
query_list = new_dataset.query
gallery_list = new_dataset.gallery

# Print the first few entries of each list for verification
print("Train List:")
for entry in train_list[:5]:
    print(entry)

print("Query List:")
for entry in query_list[:5]:
    print(entry)

print("Gallery List:")
for entry in gallery_list[:5]:
    print(entry)


=> Loaded StickerDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    42 |     6494 |       300
  query    |    42 |       42 |        34
  gallery  |    30 |       30 |        25
  ----------------------------------------
Train List:
('/home/lr/Documents/github/stick-it-ai/annotated_images/pins/ee7d6fc5-1c76-43fa-9af7-78c2a2cca343.png', 24, 71, 0)
('/home/lr/Documents/github/stick-it-ai/annotated_images/pins/68d1a874-7714-4b48-b558-55f5605a3be3.png', 24, 4, 0)
('/home/lr/Documents/github/stick-it-ai/annotated_images/pins/353d7785-7222-4b8e-9e59-f2a6606f48d7.png', 24, 30, 0)
('/home/lr/Documents/github/stick-it-ai/annotated_images/pins/3fe54f67-5465-4565-ba11-3559845f967b.png', 24, 300, 0)
('/home/lr/Documents/github/stick-it-ai/annotated_images/pins/9197b74f-a3b8-44c1-909d-4b89cd636add.png', 24, 30, 0)
Query List:
('/home/lr/Documents/github/stick-it-ai/annotated_images/pins/c768092b-37b

In [6]:
torchreid.models.show_avai_models()


['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [7]:
import pickle
from functools import partial
from collections import OrderedDict

def load_checkpoint(fpath):
    r"""Loads checkpoint.

    ``UnicodeDecodeError`` can be well handled, which means
    python2-saved files can be read from python3.

    Args:
        fpath (str): path to checkpoint.

    Returns:
        dict

    Examples::
        >>> from torchreid.utils import load_checkpoint
        >>> fpath = 'log/my_model/model.pth.tar-10'
        >>> checkpoint = load_checkpoint(fpath)
    """
    if fpath is None:
        raise ValueError('File path is None')
    fpath = osp.abspath(osp.expanduser(fpath))
    if not osp.exists(fpath):
        raise FileNotFoundError('File is not found at "{}"'.format(fpath))
    map_location = None if torch.cuda.is_available() else 'cpu'
    try:
        checkpoint = torch.load(fpath, map_location=map_location, weights_only=False)
    except UnicodeDecodeError:
        pickle.load = partial(pickle.load, encoding="latin1")
        pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")
        checkpoint = torch.load(
            fpath, pickle_module=pickle, map_location=map_location,weights_only=False
        )
    except Exception:
        print('Unable to load checkpoint from "{}"'.format(fpath))
        raise
    return checkpoint

def load_pretrained_weights(model, weight_path):
    r"""Loads pretrianed weights to model.

    Features::
        - Incompatible layers (unmatched in name or size) will be ignored.
        - Can automatically deal with keys containing "module.".

    Args:
        model (nn.Module): network model.
        weight_path (str): path to pretrained weights.

    Examples::
        >>> from torchreid.utils import load_pretrained_weights
        >>> weight_path = 'log/my_model/model-best.pth.tar'
        >>> load_pretrained_weights(model, weight_path)
    """
    checkpoint = load_checkpoint(weight_path)
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = checkpoint

    model_dict = model.state_dict()
    new_state_dict = OrderedDict()
    matched_layers, discarded_layers = [], []

    for k, v in state_dict.items():
        if k.startswith('module.'):
            k = k[7:] # discard module.

        if k in model_dict and model_dict[k].size() == v.size():
            new_state_dict[k] = v
            matched_layers.append(k)
        else:
            discarded_layers.append(k)

    model_dict.update(new_state_dict)
    model.load_state_dict(model_dict)

    if len(matched_layers) == 0:
        print.warn(
            'The pretrained weights "{}" cannot be loaded, '
            'please check the key names manually '
            '(** ignored and continue **)'.format(weight_path)
        )
    else:
        print(
            'Successfully loaded pretrained weights from "{}"'.
            format(weight_path)
        )
        if len(discarded_layers) > 0:
            print(
                '** The following layers are discarded '
                'due to unmatched keys or layer size: {}'.
                format(discarded_layers)
            )

In [8]:
def load_model(load_pretrained=True):

    model = torchreid.models.build_model(
                name="resnet50",
                num_classes=datamanager.num_train_pids,
        ).to(device)
    if load_pretrained:
        load_pretrained_weights(model, 'log/resnet50/model/model.pth.tar-40')
    return model
def init(model):
        optimizer = torchreid.optim.build_optimizer(
                model,
        )
        scheduler = torchreid.optim.build_lr_scheduler(
                optimizer,
                lr_scheduler="single_step",
                stepsize=20
        )

        engine = torchreid.engine.ImageSoftmaxEngine(
                datamanager,
                model,
                optimizer=optimizer,
                scheduler=scheduler,
        )

        return engine


In [9]:
def train(engine):
    engine.run(
                save_dir="log/resnet50",
                max_epoch=25,
                eval_freq=10,
                print_freq=10,
                test_only=False,
                fixbase_epoch=5,
                open_layers='classifier',
                visrank=False,
                visrank_topk=20
        )

In [10]:
def test(engine, model):
    engine.run(
        save_dir="log/resnet50",
        test_only=True,
        open_layers='classifier',
        visrank=False,
        visrank_topk=20
    )

In [12]:
model = load_model(load_pretrained=True)
engine = init(model)
# train(engine)
test(engine, model)

Successfully loaded pretrained weights from "log/resnet50/model/model.pth.tar-40"
##### Evaluating sticker_dataset (source) #####
Extracting features from query set ...
Done, obtained 42-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 30-by-2048 matrix
Speed: 0.0507 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
Note: number of gallery samples is quite small, got 30
** Results **
mAP: 37.4%
CMC curve
Rank-1  : 26.7%
Rank-5  : 46.7%
Rank-10 : 63.3%
Rank-20 : 86.7%


## Export model

In [13]:
import time
import json

base_path = 'model'
dataset = datamanager.train_loader.dataset
# Create directory if it doesn't exist
os.makedirs(base_path, exist_ok=True)
timestamp = int(time.time())

# Save model state dict
model_state_path = f'{base_path}/model_{timestamp}.pth'
torch.save(model.state_dict(), model_state_path)

# Save mapping information
model_data = {
    'group_id_to_pid': dataset.group_id_to_pid,
    'group_name_to_pid': dataset.pid_to_group_name,
    'model_info': {
        'timestamp': timestamp,
        'input_size': model.input_size if hasattr(model, 'input_size') else None,
        'output_size': model.output_size if hasattr(model, 'output_size') else None,
        'model_type': model.__class__.__name__
    }
}

# Save to JSON
json_path = model_state_path.replace('.pth', '.json')
with open(json_path, 'w') as json_file:
    json.dump(model_data, json_file)

print(f"Model saved to {model_state_path}")
print(f"Mapping information saved to {json_path}")


Model saved to model/model_1744450299.pth
Mapping information saved to model/model_1744450299.json


In [ ]:
model_path = 'model/model_1744379168.pth'
if json_path is None:
    json_path = model_path.replace('.pth', '.json')

# Load mapping information
with open(json_path, 'r') as json_file:
    model_data = json.load(json_file)

group_id_to_pid = model_data['group_id_to_pid']


model2 = torchreid.models.build_model(
    name="resnet50", 
    num_classes=len(group_id_to_pid),
).to(device)

# Load model state
model2.load_state_dict(torch.load(model_path, map_location=device))
model2.to(device)
model2.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'model/model_1744379168.pth'

In [ ]:
import gc
del model
gc.collect()
torch.cuda.empty_cache()
